# Modele:

Dla każdego z 3 konceptów (pelikany, flamingi, gęsi) mamy dane przepuszczone już przez pipeline (CLIP+SAE). Chcemy na nich zbudować 3 proste modele:

1) zbudowany tylko na neuronie odpowiedzialnym za koncept, który chcemy wykrywać,

2) zbudowany na wszystkich neuronach,

3) zbudowany na neuronach z największą sumą wartości dla zdjęć z danym konceptem.

Następnie porównamy te trzy podejścia w celu sprawdzenia, czy patrząc tylko na jeden neuron, który powinien być opdowiedzialny za dany koncept, jesteśmy w stanie zbudować model, który będzie dobrze decydował o tym, czy na danym zdjęciu znajduje się ten koncept. Poza tym porównamy wagi neuronów dla modelu zbudowanego na wszystkich neuronach i sprawdzimy, jakim konceptom one odpowiadają według pliku clip_ViT-L_14concept_names.csv. Pokaże to, które neurony i koncepty najbardziej decydują o tym, czy dany koncept jest obecny na zdjęciu.

## Wczytanie danych.

In [84]:
import torch

# Pelikany
X_pelican = torch.load("X_concepts_pelican.pt")
y_pelican = torch.load("y_labels_pelican.pt")

print(X_pelican.shape)
print(y_pelican.shape)

# Flamingi
X_flamingo = torch.load("X_concepts_flamingo.pt")
y_flamingo = torch.load("y_labels_flamingo.pt")

print(X_flamingo.shape)
print(y_flamingo.shape)

# Gęsi
X_goose = torch.load("X_concepts_goose.pt")
y_goose = torch.load("y_labels_goose.pt")

print(X_goose.shape)
print(y_goose.shape)


torch.Size([400, 6144])
torch.Size([400])
torch.Size([400, 6144])
torch.Size([400])
torch.Size([400, 6144])
torch.Size([400])


Sprawdźmy, ile dla danego konceptu jest danych z tym konceptem, a ile bez (czy dane są zbalansowane: powinny być, bo braliśmy wcześniej po 200 zdjęć z każdego konceptu, ale dla upewnienia).

In [104]:
with_pelicans = (y_pelican == 1).sum().item()
wo_pelicans = (y_pelican == 0).sum().item()

print(f"Liczba danych ze zbioru pelikanów z pelikanami (X_pelican): {with_pelicans}")
print(f"Liczba danych bez pelikanów ze zbioru pelikanów (X_pelican): {wo_pelicans}\n")

with_flamingos = (y_flamingo == 1).sum().item()
wo_flamingos = (y_flamingo == 0).sum().item()

print(f"Liczba danych ze zbioru flamingów z flamingami (X_flamingo): {with_flamingos}")
print(f"Liczba danych bez flamingów ze zbioru flamingów (X_flamingo): {wo_flamingos}\n")

with_geese = (y_goose == 1).sum().item()
wo_geese = (y_goose == 0).sum().item()

print(f"Liczba danych ze zbioru gęsi z gęsiami (X_goose): {with_geese}")
print(f"Liczba danych bez gęsi ze zbioru gęsi (X_goose): {wo_geese}\n")

Liczba danych ze zbioru pelikanów z pelikanami (X_pelican): 200
Liczba danych bez pelikanów ze zbioru pelikanów (X_pelican): 200

Liczba danych ze zbioru flamingów z flamingami (X_flamingo): 200
Liczba danych bez flamingów ze zbioru flamingów (X_flamingo): 200

Liczba danych ze zbioru gęsi z gęsiami (X_goose): 200
Liczba danych bez gęsi ze zbioru gęsi (X_goose): 200



## Modele dla zbioru pelikanów.

In [105]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, roc_auc_score

Podział danych na testowe i treningowe.

In [106]:
# Podział danych
X_train, X_test, y_train, y_test = train_test_split(X_pelican, y_pelican, test_size=0.2, stratify=y_pelican, random_state=42)

# neuron dla "pelican"
pelican_neuron = 1085

Sprawdźmy wartości dla neuronu, który w teorii powinien odpowiadaać za koncept pelikana.

In [107]:
X_train[y_train == 0, 1085]

tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

In [108]:
X_train[y_train == 0, 1085]

tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

Coś się nie zgadza - sprawdźmy, czy nie jest to przypadkiem zły neuron. Może jakiś inny neuron mówi nam więcej o tym, czy pelikan jest obecny na zdjęciu. Poszukujamy takiego, dla którego suma wartości neuronów dla zdjęć z pelikanami jest największa.

In [109]:
import pandas as pd

# Załaduj plik CSV z nazwami konceptów
concept_df = pd.read_csv("/content/drive/MyDrive/WB/Discover-then-Name/clip_ViT-L_14concept_names.csv",  header=None, names=["neuron_index", "concept_name"])
concept_df["neuron_index"] = concept_df["neuron_index"].astype(int)


In [110]:
import numpy as np

# Sumy aktywacji po kolumnach (neurony) tylko dla przykładów z y=1
sums_with = X_train[y_train == 1].sum(axis=0)

# Sumy po kolumnach z przykaldow bez pelikanow
sums_wo = X_train[y_train == 0].sum(axis=0)

# Indeks neuronu o najwyższej sumie
max_index = np.argmax(sums_with)
max_index = int(np.argmax(sums_with))

name_max_index = concept_df.loc[concept_df['neuron_index'] == max_index, 'concept_name'].values
name_1085 = concept_df.loc[concept_df['neuron_index'] == 1085, 'concept_name'].values

# Bezpieczne wypisanie (zabezpieczenie, gdyby brakowało w CSV)
name_max_index = name_max_index[0] if len(name_max_index) > 0 else "brak w pliku"
name_1085 = name_1085[0] if len(name_1085) > 0 else "brak w pliku"

# Wypisanie informacji
print(f"Neuron z najwyższą sumą aktywacji dla zdjęć z pelikanami: {max_index} ({name_max_index})\n")
print(f"Suma wartości tego neuronu dla zdjęć z pelikanami: {sums_with[max_index]}")
print(f"Suma wartości tego neuronu dla zdjęć bez pelikanów: {sums_wo[max_index]}\n")
print(f"Neuron z konceptem pelikana wg autorów: 1085 ({name_1085})\n")
print(f"Suma wartości tego neuronu dla zdjęć z pelikanami: {sums_with[1085]}")
print(f"Suma wartości tego neuronu dla zdjęć bez pelikanów: {sums_wo[1085]}")


Neuron z najwyższą sumą aktywacji dla zdjęć z pelikanami: 111 (birds)

Suma wartości tego neuronu dla zdjęć z pelikanami: 748.2510986328125
Suma wartości tego neuronu dla zdjęć bez pelikanów: 266.2774353027344

Neuron z konceptem pelikana wg autorów: 1085 (pelican)

Suma wartości tego neuronu dla zdjęć z pelikanami: 0.0
Suma wartości tego neuronu dla zdjęć bez pelikanów: 0.0


Zobaczmy, jak wyglądają wartości neuronu odpowiadającego za koncept 'birds'.

In [111]:
X_train[y_train == 0, 111]

tensor([0.0000, 1.1120, 4.2199, 1.1682, 1.1223, 0.1907, 0.7005, 0.6303, 0.4905,
        0.0000, 0.1484, 5.6731, 0.5801, 0.3283, 0.3541, 0.2391, 0.6215, 0.0000,
        0.6891, 5.2362, 0.3435, 0.2528, 0.4170, 0.0000, 0.0000, 0.0000, 0.7363,
        0.0000, 1.4463, 5.7929, 0.4367, 0.0000, 4.7334, 0.0854, 0.0000, 4.8739,
        0.3427, 0.0000, 0.3132, 1.3885, 1.3202, 0.5010, 4.4508, 1.7178, 0.0000,
        0.9700, 0.2841, 5.6986, 0.9745, 4.7263, 0.4672, 0.0000, 1.0944, 0.1951,
        2.5009, 3.7908, 0.9070, 0.0000, 0.0219, 1.4890, 3.9711, 0.8972, 0.0771,
        1.9473, 4.8973, 5.2224, 0.0429, 0.0000, 1.6894, 1.4184, 4.8445, 0.3620,
        0.1401, 1.8411, 0.0000, 4.5752, 0.7314, 0.0000, 5.3931, 4.8907, 1.2875,
        5.4297, 5.3211, 0.9916, 1.2021, 4.6208, 5.0872, 0.8224, 1.9143, 1.4994,
        1.0189, 0.6252, 0.4549, 0.1927, 0.7361, 4.9892, 0.9449, 6.0994, 0.0000,
        0.0086, 4.8314, 0.0000, 4.7440, 0.2613, 0.0000, 0.0377, 4.1159, 0.3147,
        0.8690, 0.0000, 0.6592, 0.2449, 

In [112]:
X_train[y_train == 1, 111]

tensor([4.7937, 4.8129, 5.5702, 5.0952, 4.1034, 4.9550, 4.5728, 3.5800, 5.2656,
        4.5237, 5.4848, 4.8634, 4.9187, 4.8093, 3.7721, 4.5295, 4.5737, 4.2526,
        4.0558, 4.7318, 4.7386, 5.2141, 5.3047, 4.9631, 4.0349, 5.3726, 5.7222,
        4.1159, 4.7866, 4.1084, 4.6997, 4.9584, 4.7285, 5.1554, 5.2639, 5.0703,
        5.4352, 4.8604, 4.6480, 4.9353, 5.4228, 5.5191, 4.8642, 4.8221, 4.8656,
        4.8986, 5.5939, 4.0814, 4.9940, 5.0489, 4.8668, 4.5827, 5.0706, 4.6171,
        5.0728, 4.9306, 5.3915, 5.3038, 5.2583, 5.2578, 3.9793, 5.0702, 4.7605,
        4.3923, 4.4375, 4.2072, 4.9728, 4.7340, 4.6938, 4.3595, 3.3883, 4.3686,
        4.5433, 4.7742, 4.7632, 4.4706, 3.9150, 4.6783, 4.8172, 4.6018, 3.5131,
        4.3222, 4.5202, 5.0122, 4.4618, 4.2244, 5.2573, 4.1716, 4.0892, 4.2365,
        4.4026, 4.6962, 5.1281, 4.2950, 4.1275, 4.5719, 5.0115, 3.4847, 5.3574,
        4.8737, 4.4896, 4.6843, 5.2600, 5.0605, 4.4088, 4.8249, 4.8969, 4.9513,
        4.2810, 4.8959, 5.3959, 5.2887, 

Model liniowy na neuronie odpowiadającym za koncept pelikana:

In [113]:
# Model na neuronie 1085
X_train_single = X_train[:, pelican_neuron].reshape(-1, 1)
X_test_single = X_test[:, pelican_neuron].reshape(-1, 1)
clf_single = LogisticRegression().fit(X_train_single, y_train)
preds_single = clf_single.predict(X_test_single)
acc_single = accuracy_score(y_test, preds_single)
roc_single = roc_auc_score(y_test, clf_single.predict_proba(X_test_single)[:, 1])

Regresja logistyczna dla wszystkich neuronów:

In [114]:
# model
clf_full = LogisticRegression(penalty="l1", solver="liblinear", max_iter=1000).fit(X_train, y_train)
preds_full = clf_full.predict(X_test)
acc_full = accuracy_score(y_test, preds_full)
roc_full = roc_auc_score(y_test, clf_full.predict_proba(X_test)[:, 1])

Model zbudowany na neuronach z największą sumą wartości dla zdjęć z danym konceptem (znajdźmy te neurony).

In [115]:
# Suma wartości dla każdego neuronu tylko na danych z pelikanami
pel_sum = X_pelican[y_pelican == 1].sum(dim=0)
# Neurony o największej sumie aktywacji
top_k = torch.topk(pel_sum, k=5)
top_neurons = top_k.indices.tolist()


print("Top aktywne neurony (pelikan):", top_neurons)

print("\nNazwy konceptów dla top 5 neuronów:")
for idx in top_neurons:
    row = concept_df.loc[concept_df["neuron_index"] == idx]
    name = row["concept_name"].values[0] if not row.empty else "brak w pliku"
    print(f"Neuron {idx}: {name}")


Top aktywne neurony (pelikan): [111, 1019, 3986, 352, 3318]

Nazwy konceptów dla top 5 neuronów:
Neuron 111: birds
Neuron 1019: wildlife
Neuron 3986: pic
Neuron 352: canoeing
Neuron 3318: beach


In [116]:
# Podział danych
X_top = X_pelican[:, top_neurons]
X_train, X_test, y_train, y_test = train_test_split(X_top, y_pelican, test_size=0.2, stratify=y_pelican, random_state=42)

# Trening modelu
clf_top = LogisticRegression(max_iter=1000)
clf_top.fit(X_train, y_train)
preds = clf_top.predict(X_test)

# Ocena
acc_top = accuracy_score(y_test, preds)
roc_top = roc_auc_score(y_test, clf_top.predict_proba(X_test)[:, 1])

## Wyniki dla zbioru pelikanów:

In [117]:
pelican_results = {
    "naming_neuron_1085": {"accuracy": acc_single, "roc_auc": roc_single},
    "logistic_regression_full": {"accuracy": acc_full, "roc_auc": roc_full},
    "top_5_neurons": {"accuracy": acc_top, "roc_auc": roc_top}
}

In [118]:
print(f"Pelican results: {pelican_results}")

Pelican results: {'naming_neuron_1085': {'accuracy': 0.5, 'roc_auc': np.float64(0.5)}, 'logistic_regression_full': {'accuracy': 0.9375, 'roc_auc': np.float64(0.9962500000000001)}, 'top_5_neurons': {'accuracy': 0.9375, 'roc_auc': np.float64(0.935625)}}


In [119]:
results_df_pelican = pd.DataFrame.from_dict({
    "Neuron 1085 (pelican)": pelican_results["naming_neuron_1085"],
    "Logistic Regression (full vector)": pelican_results["logistic_regression_full"],
    "Top 5 Neurons": pelican_results["top_5_neurons"]
}, orient="index")

display(results_df_pelican)

,accuracy,roc_auc
Neuron 1085 (pelican),0.5000,0.500000
Logistic Regression (full vector),0.9375,0.996250
Top 5 Neurons,0.9375,0.935625


## Porównanie wag neuronów dla modelu zbudowanego na wszystkich neuronach.

In [120]:
top_weights = clf_full.coef_[0].argsort()[::-1]

# Wyświetl 10 neuronów z najwyższą wagą + ich nazwy
print("Top 10 neuronów według wag modelu:")
for idx in top_weights[:10]:
    # Znajdź nazwę konceptu
    match = concept_df.loc[concept_df['neuron_index'] == idx, 'concept_name']
    concept_name = match.values[0] if len(match) > 0 else "brak nazwy"

    print(f"Neuron {idx}, Waga: {clf_full.coef_[0][idx]:.4f}, Koncept: {concept_name}")

Top 10 neuronów według wag modelu:
Neuron 937, Waga: 2.4446, Koncept: pd
Neuron 2291, Waga: 1.0192, Koncept: spiderman
Neuron 2836, Waga: 0.9608, Koncept: bloodhound
Neuron 307, Waga: 0.9474, Koncept: florida
Neuron 3318, Waga: 0.7292, Koncept: beach
Neuron 962, Waga: 0.7217, Koncept: joanna
Neuron 352, Waga: 0.2308, Koncept: canoeing
Neuron 111, Waga: 0.1315, Koncept: birds
Neuron 1903, Waga: 0.1313, Koncept: park
Neuron 2041, Waga: 0.0000, Koncept: tulip


In [121]:
print("\nBottom 10 neuronów według wag modelu:")
for idx in top_weights[-10:]:
    match = concept_df.loc[concept_df['neuron_index'] == idx, 'concept_name']
    concept_name = match.values[0] if len(match) > 0 else "brak nazwy"

    print(f"Neuron {idx}, Waga: {clf_full.coef_[0][idx]:.4f}, Koncept: {concept_name}")



Bottom 10 neuronów według wag modelu:
Neuron 6111, Waga: -0.4781, Koncept: juvenile
Neuron 3307, Waga: -0.5115, Koncept: insects
Neuron 2557, Waga: -0.5325, Koncept: grenadines
Neuron 2766, Waga: -0.5697, Koncept: underwater
Neuron 316, Waga: -0.6166, Koncept: dvd
Neuron 4894, Waga: -0.6289, Koncept: flowers
Neuron 4900, Waga: -0.7371, Koncept: halloween
Neuron 1416, Waga: -0.8020, Koncept: backpacking
Neuron 548, Waga: -0.8632, Koncept: turtles
Neuron 5685, Waga: -1.6309, Koncept: hare


In [122]:
# 3. Wypisz wagę neuronu klasy
weight_pelikan = clf_full.coef_[0][pelican_neuron]
pelican_name_match = concept_df.loc[concept_df['neuron_index'] == pelican_neuron, 'concept_name']
pelican_concept_name = pelican_name_match.values[0] if len(pelican_name_match) > 0 else "brak nazwy"

print(f"Neuron przypisany do klasy 'pelican': {pelican_neuron}")
print(f"  → Waga w modelu: {weight_pelikan:.4f}")
print(f"  → Nazwa konceptu: {pelican_concept_name}")

# 4. Znajdź neuron o najwyższej wadze
top_idx = np.argmax(clf_full.coef_[0])
top_weight = clf_full.coef_[0][top_idx]
top_name_match = concept_df.loc[concept_df['neuron_index'] == top_idx, 'concept_name']
top_concept_name = top_name_match.values[0] if len(top_name_match) > 0 else "brak nazwy"

print(f"\nNeuron z największą wagą:")
print(f"  → Neuron: {top_idx}")
print(f"  → Waga: {top_weight:.4f}")
print(f"  → Nazwa konceptu: {top_concept_name}")

Neuron przypisany do klasy 'pelican': 1085
  → Waga w modelu: 0.0000
  → Nazwa konceptu: pelican

Neuron z największą wagą:
  → Neuron: 937
  → Waga: 2.4446
  → Nazwa konceptu: pd


## Porównanie wag neuronów dla modelu zbudowanego na neuronach z najwyższą sumą dla zdjęc z pelikanami.

In [123]:
top_weights = clf_top.coef_[0].argsort()[::-1]

# Wyświetl 10 neuronów z najwyższą wagą + ich nazwy
print("Wszystkie neurony według wag modelu:")
for local_idx in top_weights:
    global_idx = top_neurons[local_idx]  # oryginalny indeks neuronu
    match = concept_df.loc[concept_df['neuron_index'] == global_idx, 'concept_name']
    concept_name = match.values[0] if len(match) > 0 else "brak nazwy"

    print(f"Neuron (w concept names): {global_idx}, Neuron w tym modelu: {local_idx}, Waga: {clf_top.coef_[0][local_idx]:.4f}, Koncept: {concept_name}")

Wszystkie neurony według wag modelu:
Neuron (w concept names): 352, Neuron w tym modelu: 3, Waga: 1.6925, Koncept: canoeing
Neuron (w concept names): 3318, Neuron w tym modelu: 4, Waga: 1.6772, Koncept: beach
Neuron (w concept names): 111, Neuron w tym modelu: 0, Waga: 1.4003, Koncept: birds
Neuron (w concept names): 1019, Neuron w tym modelu: 1, Waga: 0.8460, Koncept: wildlife
Neuron (w concept names): 3986, Neuron w tym modelu: 2, Waga: -0.0586, Koncept: pic


## Modele dla zbioru flamingów.

Podział danych na testowe i treningowe.

In [67]:
# Podział danych
X_train, X_test, y_train, y_test = train_test_split(X_flamingo, y_flamingo, test_size=0.2, stratify=y_flamingo, random_state=42)

# neuron dla "flamingo"
flamingo_neuron = 2347


Sprawdźmy wartości dla neuronu, który w teorii powinien odpowiadać za koncept flaminga.

In [68]:
X_train[y_train == 1, 2347]

tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

In [69]:
X_train[y_train == 0, 2347]

tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

Znowu same zera, poszukajmy dla którego neuronu suma wartości dla zdjęć z flamingiem jest największa.

In [70]:
# Sumy aktywacji po kolumnach (neurony) tylko dla przykładów z y=1
sums_with = X_train[y_train == 1].sum(axis=0)

# Sumy po kolumnach z przykaldow bez pelikanow
sums_wo = X_train[y_train == 0].sum(axis=0)

# Indeks neuronu o najwyższej sumie
max_index = np.argmax(sums_with)
max_index = int(np.argmax(sums_with))

name_max_index = concept_df.loc[concept_df['neuron_index'] == max_index, 'concept_name'].values
name_2347 = concept_df.loc[concept_df['neuron_index'] == 2347, 'concept_name'].values

# Bezpieczne wypisanie (zabezpieczenie, gdyby brakowało w CSV)
name_max_index = name_max_index[0] if len(name_max_index) > 0 else "brak w pliku"
name_2347 = name_2347[0] if len(name_2347) > 0 else "brak w pliku"

# Wypisanie informacji
print(f"Neuron z najwyższą sumą aktywacji dla zdjęć z pelikanami: {max_index} ({name_max_index})\n")
print(f"Suma wartości tego neuronu dla zdjęć z pelikanami: {sums_with[max_index]}")
print(f"Suma wartości tego neuronu dla zdjęć bez pelikanów: {sums_wo[max_index]}\n")
print(f"Neuron z konceptem pelikana wg autorów: 1085 ({name_2347})\n")
print(f"Suma wartości tego neuronu dla zdjęć z pelikanami: {sums_with[2347]}")
print(f"Suma wartości tego neuronu dla zdjęć bez pelikanów: {sums_wo[2347]}")


Neuron z najwyższą sumą aktywacji dla zdjęć z pelikanami: 111 (birds)

Suma wartości tego neuronu dla zdjęć z pelikanami: 650.3216552734375
Suma wartości tego neuronu dla zdjęć bez pelikanów: 343.5186462402344

Neuron z konceptem pelikana wg autorów: 1085 (flamingo)

Suma wartości tego neuronu dla zdjęć z pelikanami: 0.0
Suma wartości tego neuronu dla zdjęć bez pelikanów: 0.0


Model liniowy na neuronie odpowiadającym za koncept flaminga:

In [71]:
# Model na pojedynczym neuronie
X_train_single = X_train[:, flamingo_neuron].reshape(-1, 1)
X_test_single = X_test[:, flamingo_neuron].reshape(-1, 1)
clf_single = LogisticRegression().fit(X_train_single, y_train)
preds_single = clf_single.predict(X_test_single)
acc_single = accuracy_score(y_test, preds_single)
roc_single = roc_auc_score(y_test, clf_single.predict_proba(X_test_single)[:, 1])


Regresja logistyczna dla wszystkich neuronów:

In [72]:
# Model na pełnym wektorze
clf_full = LogisticRegression(penalty="l1", solver="liblinear", max_iter=1000).fit(X_train, y_train)
preds_full = clf_full.predict(X_test)
acc_full = accuracy_score(y_test, preds_full)
roc_full = roc_auc_score(y_test, clf_full.predict_proba(X_test)[:, 1])

Model zbudowany na neuronach z największą sumą wartości dla zdjęć z danym konceptem (znajdźmy te neurony).

In [83]:
# Suma wartości dla każdego neuronu tylko na danych z flamingami
flam_sum = X_flamingo[y_flamingo == 1].sum(dim=0)
# Neurony o największej sumie aktywacji
top_k = torch.topk(flam_sum, k=5)
top_neurons = top_k.indices.tolist()


print("Top aktywne neurony (flaming):", top_neurons)

print("\nNazwy konceptów dla top 5 neuronów:")
for idx in top_neurons:
    row = concept_df.loc[concept_df["neuron_index"] == idx]
    name = row["concept_name"].values[0] if not row.empty else "brak w pliku"
    print(f"Neuron {idx}: {name}")


Top aktywne neurony (flaming): [111, 1019, 3986, 2842, 5108]

Nazwy konceptów dla top 5 neuronów:
Neuron 111: birds
Neuron 1019: wildlife
Neuron 3986: pic
Neuron 2842: pinkworld
Neuron 5108: banner


In [74]:
# Podział danych
X_top = X_flamingo[:, top_neurons]
X_train, X_test, y_train, y_test = train_test_split(X_top, y_flamingo, test_size=0.2, stratify=y_flamingo, random_state=42)

# Trening modelu
clf_top = LogisticRegression(max_iter=1000)
clf_top.fit(X_train, y_train)
preds = clf_top.predict(X_test)

# Ocena
acc_top = accuracy_score(y_test, preds)
roc_top = roc_auc_score(y_test, clf_top.predict_proba(X_test)[:, 1])

## Wyniki dla zbioru flamingów:

In [75]:
results_flamingo = {
    "naming_neuron": {"accuracy": acc_single, "roc_auc": roc_single},
    "logistic_regression_full": {"accuracy": acc_full, "roc_auc": roc_full},
    "top_5_neurons": {"accuracy": acc_top, "roc_auc": roc_top}
}

In [76]:
print(f"Flamingo results: {results_flamingo}")

Flamingo results: {'naming_neuron': {'accuracy': 0.5, 'roc_auc': np.float64(0.5)}, 'logistic_regression_full': {'accuracy': 0.9875, 'roc_auc': np.float64(0.9981249999999999)}, 'top_5_neurons': {'accuracy': 0.975, 'roc_auc': np.float64(0.9981249999999999)}}


In [77]:
results_df_flamingo = pd.DataFrame.from_dict({
    "Neuron 1085 (flamingo)": results_flamingo["naming_neuron"],
    "Logistic Regression (full vector)": results_flamingo["logistic_regression_full"],
    "Top 5 Neurons": results_flamingo["top_5_neurons"]
}, orient="index")

display(results_df_flamingo)

,accuracy,roc_auc
Neuron 1085 (flamingo),0.5000,0.500000
Logistic Regression (full vector),0.9875,0.998125
Top 5 Neurons,0.9750,0.998125


## Porównanie wag neuronów dla modelu zbudowanego na wszystkich neuronach.

In [78]:
top_weights = clf_full.coef_[0].argsort()[::-1]

# Wyświetl 10 neuronów z najwyższą wagą + ich nazwy
print("Top 10 neuronów według wag modelu:")
for idx in top_weights[:10]:
    # Znajdź nazwę konceptu
    match = concept_df.loc[concept_df['neuron_index'] == idx, 'concept_name']
    concept_name = match.values[0] if len(match) > 0 else "brak nazwy"

    print(f"Neuron {idx}, Waga: {clf_full.coef_[0][idx]:.4f}, Koncept: {concept_name}")

Top 10 neuronów według wag modelu:
Neuron 2842, Waga: 3.0988, Koncept: pinkworld
Neuron 1627, Waga: 0.8072, Koncept: refused
Neuron 3809, Waga: 0.5213, Koncept: yellow
Neuron 3196, Waga: 0.2257, Koncept: orange
Neuron 2033, Waga: 0.0000, Koncept: parsing
Neuron 2034, Waga: 0.0000, Koncept: attic
Neuron 2035, Waga: 0.0000, Koncept: frontgate
Neuron 2036, Waga: 0.0000, Koncept: church
Neuron 2037, Waga: 0.0000, Koncept: talented
Neuron 2038, Waga: 0.0000, Koncept: kitten


In [79]:
print("\nBottom 10 neuronów według wag modelu:")
for idx in top_weights[-10:]:
    match = concept_df.loc[concept_df['neuron_index'] == idx, 'concept_name']
    concept_name = match.values[0] if len(match) > 0 else "brak nazwy"

    print(f"Neuron {idx}, Waga: {clf_full.coef_[0][idx]:.4f}, Koncept: {concept_name}")


Bottom 10 neuronów według wag modelu:
Neuron 111, Waga: -0.1556, Koncept: birds
Neuron 6111, Waga: -0.3860, Koncept: juvenile
Neuron 5978, Waga: -0.3882, Koncept: pixel
Neuron 2072, Waga: -0.3884, Koncept: tasmania
Neuron 3307, Waga: -0.5246, Koncept: insects
Neuron 835, Waga: -0.5993, Koncept: butte
Neuron 4324, Waga: -0.6969, Koncept: visitor
Neuron 5499, Waga: -0.8694, Koncept: didrex
Neuron 3576, Waga: -0.8824, Koncept: emergence
Neuron 831, Waga: -1.2961, Koncept: fishes


In [80]:
# Wypisz wagę neuronu klasy
weight_flamingo = clf_full.coef_[0][flamingo_neuron]
flamingo_name_match = concept_df.loc[concept_df['neuron_index'] == flamingo_neuron, 'concept_name']
flamingo_concept_name = flamingo_name_match.values[0] if len(flamingo_name_match) > 0 else "brak nazwy"

print(f"Neuron przypisany do klasy 'flamingo': {flamingo_neuron}")
print(f"  → Waga w modelu: {weight_flamingo:.4f}")
print(f"  → Nazwa konceptu: {flamingo_concept_name}")

# Znajdź neuron o najwyższej wadze
top_idx = np.argmax(clf_full.coef_[0])
top_weight = clf_full.coef_[0][top_idx]
top_name_match = concept_df.loc[concept_df['neuron_index'] == top_idx, 'concept_name']
top_concept_name = top_name_match.values[0] if len(top_name_match) > 0 else "brak nazwy"

print(f"\nNeuron z największą wagą:")
print(f"  → Neuron: {top_idx}")
print(f"  → Waga: {top_weight:.4f}")
print(f"  → Nazwa konceptu: {top_concept_name}")

Neuron przypisany do klasy 'flamingo': 2347
  → Waga w modelu: 0.0000
  → Nazwa konceptu: flamingo

Neuron z największą wagą:
  → Neuron: 2842
  → Waga: 3.0988
  → Nazwa konceptu: pinkworld


## Porównanie wag neuronów dla modelu zbudowanego na neuronach z najwyższą sumą dla zdjęc z pelikanami.

In [82]:
top_weights = clf_top.coef_[0].argsort()[::-1]

# Wyświetl 10 neuronów z najwyższą wagą + ich nazwy
print("Wszystkie neurony według wag modelu:")
for local_idx in top_weights:
    global_idx = top_neurons[local_idx]  # oryginalny indeks neuronu
    match = concept_df.loc[concept_df['neuron_index'] == global_idx, 'concept_name']
    concept_name = match.values[0] if len(match) > 0 else "brak nazwy"

    print(f"Neuron (w concept names): {global_idx}, Neuron w tym modelu: {local_idx}, Waga: {clf_top.coef_[0][local_idx]:.4f}, Koncept: {concept_name}")

Wszystkie neurony według wag modelu:
Neuron (w concept names): 2842, Neuron w tym modelu: 3, Waga: 3.3412, Koncept: pinkworld
Neuron (w concept names): 1019, Neuron w tym modelu: 1, Waga: 0.6896, Koncept: wildlife
Neuron (w concept names): 111, Neuron w tym modelu: 0, Waga: 0.5055, Koncept: birds
Neuron (w concept names): 3986, Neuron w tym modelu: 2, Waga: 0.1954, Koncept: pic
Neuron (w concept names): 5108, Neuron w tym modelu: 4, Waga: -0.1805, Koncept: banner


## Modele dla zbioru gęsi.

Podział danych na testowe i treningowe.

In [127]:
# Podział danych
X_train, X_test, y_train, y_test = train_test_split(X_goose, y_goose, test_size=0.2, stratify=y_goose, random_state=42)

# neuron dla "goose"
goose_neuron = 3426

Sprawdźmy wartości dla neuronu, który w teorii powinien odpowiadać za koncept gęsi.

In [128]:
X_train[y_train == 1, goose_neuron]

tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

In [129]:
X_train[y_train == 0, goose_neuron]

tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

Dziwne, ale znowu same zera. Poszukajmy dla którego neuronu suma wartości dla zdjęć z gęsiami jest największa.

In [131]:
# Sumy aktywacji po kolumnach (neurony) tylko dla przykładów z y=1
sums_with = X_train[y_train == 1].sum(axis=0)
sums_wo = X_train[y_train == 0].sum(axis=0)
# Indeks neuronu o najwyższej sumie
max_index = np.argmax(sums_with)
max_index = int(np.argmax(sums_with))

# Pobranie nazw konceptów
name_max_index = concept_df.loc[concept_df['neuron_index'] == max_index, 'concept_name'].values
name_3426 = concept_df.loc[concept_df['neuron_index'] == 3426, 'concept_name'].values

# Bezpieczne wypisanie (zabezpieczenie, gdyby brakowało w CSV)
name_max_index = name_max_index[0] if len(name_max_index) > 0 else "brak w pliku"
name_3426 = name_3426[0] if len(name_3426) > 0 else "brak w pliku"

# Wypisanie wyników
print(f"Neuron z najwyższą sumą aktywacji dla zdjęć z gęsiami: {max_index} ({name_max_index})")
print(f"Suma wartości tego neuronu dla zdjęć z gęsiami: {sums_with[max_index]}")
print(f"Suma wartości tego neuronu dla zdjęć bez gęsi: {sums_wo[max_index]}\n")

print(f"Neuron z konceptem gęsi wg autorów: 3426 ({name_3426})")
print(f"Suma wartości tego neuronu dla zdjęć z gęsiami: {sums_with[3426]}")
print(f"Suma wartości tego neuronu dla zdjęć bez gęsi: {sums_wo[3426]}")


Neuron z najwyższą sumą aktywacji dla zdjęć z gęsiami: 111 (birds)
Suma wartości tego neuronu dla zdjęć z gęsiami: 755.861083984375
Suma wartości tego neuronu dla zdjęć bez gęsi: 295.38323974609375

Neuron z konceptem gęsi wg autorów: 3426 (goose)
Suma wartości tego neuronu dla zdjęć z gęsiami: 0.0
Suma wartości tego neuronu dla zdjęć bez gęsi: 0.0


Znowu ptaki, czego można było się spodziewać.

Model liniowy na neuronie odpowiadającym za koncept gęsi:

In [132]:
# Model na pojedynczym neuronie
X_train_single = X_train[:, goose_neuron].reshape(-1, 1)
X_test_single = X_test[:, goose_neuron].reshape(-1, 1)
clf_single = LogisticRegression().fit(X_train_single, y_train)
preds_single = clf_single.predict(X_test_single)
acc_single = accuracy_score(y_test, preds_single)
roc_single = roc_auc_score(y_test, clf_single.predict_proba(X_test_single)[:, 1])

Regresja logistyczna dla wszystkich neuronów:

In [133]:
# Model na pełnym wektorze
clf_full = LogisticRegression(penalty="l1", solver="liblinear", max_iter=1000).fit(X_train, y_train)
preds_full = clf_full.predict(X_test)
acc_full = accuracy_score(y_test, preds_full)
roc_full = roc_auc_score(y_test, clf_full.predict_proba(X_test)[:, 1])

Model zbudowany na neuronach z największą sumą wartości dla zdjęć z danym konceptem (znajdźmy te neurony).

In [134]:
# Suma wartości dla każdego neuronu tylko na danych z gęsiami
goose_sum = X_goose[y_goose == 1].sum(dim=0)
# Neurony o największej sumie aktywacji
top_k = torch.topk(goose_sum, k=5)
top_neurons = top_k.indices.tolist()


print("Top aktywne neurony (goose):", top_neurons)

print("\nNazwy konceptów dla top 5 neuronów:")
for idx in top_neurons:
    row = concept_df.loc[concept_df["neuron_index"] == idx]
    name = row["concept_name"].values[0] if not row.empty else "brak w pliku"
    print(f"Neuron {idx}: {name}")


Top aktywne neurony (goose): [111, 3986, 352, 1019, 184]

Nazwy konceptów dla top 5 neuronów:
Neuron 111: birds
Neuron 3986: pic
Neuron 352: canoeing
Neuron 1019: wildlife
Neuron 184: audi


In [135]:
# Podział danych
X_top = X_goose[:, top_neurons]
X_train, X_test, y_train, y_test = train_test_split(X_top, y_goose, test_size=0.2, stratify=y_goose, random_state=42)

# Trening modelu
clf_top = LogisticRegression(max_iter=1000)
clf_top.fit(X_train, y_train)
preds = clf_top.predict(X_test)

# Ocena
acc_top = accuracy_score(y_test, preds)
roc_top = roc_auc_score(y_test, clf_top.predict_proba(X_test)[:, 1])

## Wyniki dla zbioru gęsi:

In [136]:
results_goose = {
    "naming_neuron": {"accuracy": acc_single, "roc_auc": roc_single},
    "logistic_regression_full": {"accuracy": acc_full, "roc_auc": roc_full},
    "top_5_neurons": {"accuracy": acc_top, "roc_auc": roc_top}
}

In [137]:
print(f"Goose results: {results_goose}")

Goose results: {'naming_neuron': {'accuracy': 0.5, 'roc_auc': np.float64(0.5)}, 'logistic_regression_full': {'accuracy': 1.0, 'roc_auc': np.float64(1.0)}, 'top_5_neurons': {'accuracy': 0.975, 'roc_auc': np.float64(1.0)}}


In [138]:
results_df_goose = pd.DataFrame.from_dict({
    "Neuron 1085 (goose)": results_goose["naming_neuron"],
    "Logistic Regression (full vector)": results_goose["logistic_regression_full"],
    "Top 5 Neurons": results_goose["top_5_neurons"]
}, orient="index")

display(results_df_goose)

,accuracy,roc_auc
Neuron 1085 (goose),0.500,0.5
Logistic Regression (full vector),1.000,1.0
Top 5 Neurons,0.975,1.0


## Porównanie wag neuronów dla modelu zbudowanego na wszystkich neuronach.

In [139]:
# Wypisz wagę neuronu klasy
weight_goose = clf_full.coef_[0][goose_neuron]
goose_name_match = concept_df.loc[concept_df['neuron_index'] == goose_neuron, 'concept_name']
goose_concept_name = goose_name_match.values[0] if len(goose_name_match) > 0 else "brak nazwy"

print(f"Neuron przypisany do klasy 'goose': {goose_neuron}")
print(f"  → Waga w modelu: {weight_goose:.4f}")
print(f"  → Nazwa konceptu: {goose_concept_name}")

# Znajdź neuron o najwyższej wadze
top_idx = np.argmax(clf_full.coef_[0])
top_weight = clf_full.coef_[0][top_idx]
top_name_match = concept_df.loc[concept_df['neuron_index'] == top_idx, 'concept_name']
top_concept_name = top_name_match.values[0] if len(top_name_match) > 0 else "brak nazwy"

print(f"\nNeuron z największą wagą:")
print(f"  → Neuron: {top_idx}")
print(f"  → Waga: {top_weight:.4f}")
print(f"  → Nazwa konceptu: {top_concept_name}")

Neuron przypisany do klasy 'goose': 3426
  → Waga w modelu: 0.0000
  → Nazwa konceptu: goose

Neuron z największą wagą:
  → Neuron: 1721
  → Waga: 1.4610
  → Nazwa konceptu: huskies


In [140]:
top_weights = clf_full.coef_[0].argsort()[::-1]

# Wyświetl 10 neuronów z najwyższą wagą + ich nazwy
print("Top 10 neuronów według wag modelu:")
for idx in top_weights[:10]:
    # Znajdź nazwę konceptu
    match = concept_df.loc[concept_df['neuron_index'] == idx, 'concept_name']
    concept_name = match.values[0] if len(match) > 0 else "brak nazwy"

    print(f"Neuron {idx}, Waga: {clf_full.coef_[0][idx]:.4f}, Koncept: {concept_name}")

Top 10 neuronów według wag modelu:
Neuron 1721, Waga: 1.4610, Koncept: huskies
Neuron 3124, Waga: 1.4371, Koncept: cows
Neuron 184, Waga: 1.1564, Koncept: audi
Neuron 352, Waga: 0.8777, Koncept: canoeing
Neuron 3008, Waga: 0.5603, Koncept: sunflower
Neuron 1086, Waga: 0.2661, Koncept: silica
Neuron 111, Waga: 0.0955, Koncept: birds
Neuron 1072, Waga: 0.0510, Koncept: rewrite
Neuron 4371, Waga: 0.0360, Koncept: octet
Neuron 2062, Waga: 0.0000, Koncept: cheyenne


In [141]:
print("\nBottom 10 neuronów według wag modelu:")
for idx in top_weights[-10:]:
    match = concept_df.loc[concept_df['neuron_index'] == idx, 'concept_name']
    concept_name = match.values[0] if len(match) > 0 else "brak nazwy"

    print(f"Neuron {idx}, Waga: {clf_full.coef_[0][idx]:.4f}, Koncept: {concept_name}")


Bottom 10 neuronów według wag modelu:
Neuron 1019, Waga: -0.2940, Koncept: wildlife
Neuron 6111, Waga: -0.3992, Koncept: juvenile
Neuron 548, Waga: -0.5757, Koncept: turtles
Neuron 5230, Waga: -0.5794, Koncept: rainbow
Neuron 3640, Waga: -0.7452, Koncept: pint
Neuron 2072, Waga: -0.8507, Koncept: tasmania
Neuron 5499, Waga: -0.8750, Koncept: didrex
Neuron 3307, Waga: -1.0218, Koncept: insects
Neuron 831, Waga: -1.2258, Koncept: fishes
Neuron 2792, Waga: -1.6311, Koncept: red


## Porównanie wag neuronów dla modelu zbudowanego na neuronach z najwyższą sumą dla zdjęć z gęsiami.

In [142]:
top_weights = clf_top.coef_[0].argsort()[::-1]

# Wyświetl 10 neuronów z najwyższą wagą + ich nazwy
print("Wszystkie neurony według wag modelu:")
for local_idx in top_weights:
    global_idx = top_neurons[local_idx]  # oryginalny indeks neuronu
    match = concept_df.loc[concept_df['neuron_index'] == global_idx, 'concept_name']
    concept_name = match.values[0] if len(match) > 0 else "brak nazwy"

    print(f"Neuron (w concept names): {global_idx}, Neuron w tym modelu: {local_idx}, Waga: {clf_top.coef_[0][local_idx]:.4f}, Koncept: {concept_name}")

Wszystkie neurony według wag modelu:
Neuron (w concept names): 184, Neuron w tym modelu: 4, Waga: 2.2035, Koncept: audi
Neuron (w concept names): 352, Neuron w tym modelu: 2, Waga: 1.5191, Koncept: canoeing
Neuron (w concept names): 111, Neuron w tym modelu: 0, Waga: 1.1843, Koncept: birds
Neuron (w concept names): 3986, Neuron w tym modelu: 1, Waga: 0.0146, Koncept: pic
Neuron (w concept names): 1019, Neuron w tym modelu: 3, Waga: -0.9996, Koncept: wildlife
